In [5]:
import pandas as pd
import joblib
import numpy as np
from datetime import datetime

print("="*80)
print("🏦 CREDIT SCORING SYSTEM - XGBoost Model Demo")
print("="*80)
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

🏦 CREDIT SCORING SYSTEM - XGBoost Model Demo
📅 Date: 2025-11-13 01:03:14



In [6]:
# Load the trained XGBoost model and optimal threshold
print("Loading XGBoost model...")

xgb_model = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\lgb_model_optimized.pkl")
metadata = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\ensemble_comparison_metadata.pkl")

# Get XGBoost threshold
threshold = metadata['models']['xgboost']['threshold']

print("✅ Model loaded successfully!")
print(f"✅ Optimal threshold: {threshold:.3f}")
print(f"✅ Total features: {len(xgb_model.feature_names_in_)}\n")

Loading XGBoost model...
✅ Model loaded successfully!
✅ Optimal threshold: 0.860
✅ Total features: 64



In [7]:
# User-friendly feature configuration
FEATURE_CONFIG = [
    {
        "field": "age_years",
        "label": "🎂 Age",
        "question": "How old is the customer?",
        "description": "Customer's age in years. Most applicants are between 25-60 years old.",
        "unit": "years",
        "range": (18, 75),
        "default": 35,
        "examples": "Example: 30, 45, 52"
    },
    {
        "field": "employment_years",
        "label": "💼 Work Experience",
        "question": "How many years has the customer been working?",
        "description": "Total years of employment (including all jobs, can be non-consecutive).",
        "unit": "years",
        "range": (0, 50),
        "default": 5,
        "examples": "Example: 3 (3 years), 10 (10 years), 0 (fresh graduate)"
    },
    {
        "field": "AMT_INCOME_TOTAL",
        "label": "💰 Annual Income",
        "question": "What is the customer's annual income?",
        "description": "Total yearly income from all sources (salary, business, etc.)",
        "unit": "USD",
        "range": (10000, 1000000),
        "default": 50000,
        "examples": "Example: 30000, 60000, 120000",
        "display_format": "${:,.0f}"
    },
    {
        "field": "AMT_CREDIT",
        "label": "🏦 Requested Loan Amount",
        "question": "How much money does the customer want to borrow?",
        "description": "The total loan amount requested by the customer.",
        "unit": "USD",
        "range": (5000, 500000),
        "default": 150000,
        "examples": "Example: 50000, 150000, 300000",
        "display_format": "${:,.0f}"
    },
    {
        "field": "credit_income_ratio",
        "label": "📊 Loan-to-Income Ratio",
        "question": "Loan amount compared to annual income",
        "description": "How many times bigger is the loan compared to yearly income. Lower is better.\n"
                      "  • Below 2: Low risk (loan is 2x or less than income)\n"
                      "  • 2-5: Medium risk\n"
                      "  • Above 5: High risk (loan is much bigger than income)",
        "unit": "times",
        "range": (0, 10),
        "default": 2.5,
        "examples": "Example: 1.5 (loan = 1.5x income), 3.0 (loan = 3x income)",
        "auto_calculate": True,
        "formula": "AMT_CREDIT / AMT_INCOME_TOTAL"
    },
    {
        "field": "total_utilization",
        "label": "💳 Credit Usage Rate",
        "question": "How much credit is currently being used?",
        "description": "Percentage of credit limit being used across all cards/loans.\n"
                      "  • 0-30%: Excellent (low usage)\n"
                      "  • 30-50%: Good (moderate usage)\n"
                      "  • 50-80%: Fair (high usage)\n"
                      "  • 80-100%: Poor (maxed out)\n"
                      "  • >100%: Critical (over limit)",
        "unit": "%",
        "range": (0, 200),
        "default": 45,
        "examples": "Example: 30 (using 30%), 75 (using 75%), 120 (over limit by 20%)",
        "cast": lambda v: float(v) / 100,
        "display_format": "{:.0f}%"
    },
    {
        "field": "dpd_mean",
        "label": "⏰ Average Days Late",
        "question": "On average, how many days late are payments?",
        "description": "Average number of days customer pays AFTER the due date (last 12 months).\n"
                      "  • 0 days: Always on time (excellent)\n"
                      "  • 1-5 days: Occasionally late (good)\n"
                      "  • 6-15 days: Frequently late (fair)\n"
                      "  • 15-30 days: Very late (poor)\n"
                      "  • >30 days: Seriously delinquent (bad)",
        "unit": "days",
        "range": (0, 90),
        "default": 3,
        "examples": "Example: 0 (always on time), 5 (few days late), 30 (month late)"
    },
    {
        "field": "cc_avg_utilization",
        "label": "💳 Credit Card Usage",
        "question": "What percentage of credit card limit is being used?",
        "description": "How much of the credit card limit is currently spent.\n"
                      "  • 0-10%: Excellent (barely using cards)\n"
                      "  • 10-30%: Good (healthy usage)\n"
                      "  • 30-60%: Fair (moderate usage)\n"
                      "  • 60-90%: Poor (high usage)\n"
                      "  • >90%: Critical (almost maxed out)",
        "unit": "%",
        "range": (0, 100),
        "default": 42,
        "examples": "Example: 25 (using 25%), 60 (using 60%), 95 (almost full)",
        "cast": lambda v: float(v) / 100,
        "display_format": "{:.0f}%"
    },
    {
        "field": "NAME_EDUCATION_TYPE_Higher education",
        "label": "🎓 Education Level",
        "question": "Does the customer have higher education (college/university)?",
        "description": "Higher education includes: Bachelor's degree, Master's, PhD, or equivalent.",
        "unit": "",
        "range": (0, 1),
        "default": 1,
        "examples": "Enter 1 for YES, 0 for NO",
        "cast": int,
        "is_binary": True
    },
    {
        "field": "NAME_INCOME_TYPE_Working",
        "label": "💼 Employment Status",
        "question": "Is the customer currently employed (working)?",
        "description": "Select YES if customer has a regular job. Select NO for self-employed, retired, student, unemployed.",
        "unit": "",
        "range": (0, 1),
        "default": 1,
        "examples": "Enter 1 for YES (employed), 0 for NO (other)",
        "cast": int,
        "is_binary": True
    },
]

print("✅ Feature configuration loaded!")
print(f"✅ Total questions to ask: {len([c for c in FEATURE_CONFIG if not c.get('auto_calculate')])}\n")

✅ Feature configuration loaded!
✅ Total questions to ask: 9



In [8]:
def ask_value(cfg: dict, context: dict = None) -> float:
    """
    Ask user for input with validation and helpful prompts
    
    Parameters:
    -----------
    cfg : dict
        Configuration dictionary for the field
    context : dict
        Previously collected values (for auto-calculation)
    
    Returns:
    --------
    float : The validated value
    """
    # Check if this field should be auto-calculated
    if cfg.get("auto_calculate") and context:
        formula = cfg.get("formula", "")
        if formula:
            try:
                # Simple formula evaluation
                if "/" in formula:
                    parts = formula.split("/")
                    numerator = context.get(parts[0].strip())
                    denominator = context.get(parts[1].strip())
                    if numerator and denominator:
                        value = numerator / denominator
                        print(f"\n{cfg['label']}")
                        print(f"  ℹ️  Auto-calculated: {value:.2f} {cfg.get('unit', '')}")
                        print(f"  📐 Formula: {formula}")
                        return value
            except:
                pass
    
    cast = cfg.get("cast", float)
    low, high = cfg.get("range", (None, None))
    default = cfg.get("default")
    unit = cfg.get("unit", "")
    is_binary = cfg.get("is_binary", False)
    
    # Build prompt
    print("\n" + "─"*60)
    print(f"{cfg['label']}")
    print(f"❓ {cfg['question']}")
    
    if is_binary:
        prompt = f"Enter (1=YES, 0=NO)"
    else:
        prompt = f"Enter value"
        if unit:
            prompt += f" in {unit}"
        if low is not None and high is not None:
            prompt += f" (range: {low}-{high})"
    
    if default is not None:
        prompt += f" [default: {default}]"
    
    prompt += "\n💡 Type '?' for more info, or press Enter for default: "
    
    while True:
        raw = input(prompt).strip()
        
        # Show help
        if raw == "?":
            print("\n📖 Help:")
            print(f"   {cfg['description']}")
            if cfg.get('examples'):
                print(f"   {cfg['examples']}")
            print()
            continue
        
        # Use default
        if raw == "":
            if default is None:
                print("❌ This field is required. Please enter a value or type '?' for help.")
                continue
            value = default
            print(f"✓ Using default: {default}")
        else:
            # Parse input
            raw = raw.replace(",", "").replace("$", "").strip()
            try:
                value = cast(raw)
            except ValueError:
                print(f"❌ Invalid input. Please enter a valid number.")
                continue
        
        # Validate range
        if low is not None and value < low:
            print(f"❌ Value too low. Minimum is {low}.")
            continue
        if high is not None and value > high:
            print(f"❌ Value too high. Maximum is {high}.")
            continue
        
        # Display confirmation
        display_format = cfg.get("display_format", "{}")
        if is_binary:
            display_value = "YES" if value == 1 else "NO"
        else:
            display_value = display_format.format(value)
        print(f"✅ Confirmed: {display_value}")
        
        return value

print("📝 User input function ready!")

📝 User input function ready!


In [9]:
# Interactive data collection
print("\n" + "="*80)
print("📝 CUSTOMER INFORMATION FORM")
print("="*80)
print("\n💡 Tips:")
print("  • Press Enter to use default values (shown in brackets)")
print("  • Type '?' at any question to see detailed explanation")
print("  • All values will be validated automatically")
print("\nLet's start!\n")

# Collect all values
values = {}
for cfg in FEATURE_CONFIG:
    field_value = ask_value(cfg, context=values)
    values[cfg["field"]] = field_value

print("\n" + "="*80)
print("✅ All information collected!")
print("="*80)


📝 CUSTOMER INFORMATION FORM

💡 Tips:
  • Press Enter to use default values (shown in brackets)
  • Type '?' at any question to see detailed explanation
  • All values will be validated automatically

Let's start!


────────────────────────────────────────────────────────────
🎂 Age
❓ How old is the customer?
✅ Confirmed: 18.0

────────────────────────────────────────────────────────────
💼 Work Experience
❓ How many years has the customer been working?
✅ Confirmed: 0.0

────────────────────────────────────────────────────────────
💰 Annual Income
❓ What is the customer's annual income?
✅ Confirmed: $20,000

────────────────────────────────────────────────────────────
🏦 Requested Loan Amount
❓ How much money does the customer want to borrow?
✅ Confirmed: $40,000

📊 Loan-to-Income Ratio
  ℹ️  Auto-calculated: 2.00 times
  📐 Formula: AMT_CREDIT / AMT_INCOME_TOTAL

────────────────────────────────────────────────────────────
💳 Credit Usage Rate
❓ How much credit is currently being used?
✅ Co

In [13]:
# Prepare data for prediction
print("\n🔄 Preparing data for prediction...")

# Create DataFrame with collected values
sample = pd.DataFrame([values])

# Fill missing features with 0 (XGBoost will handle missing features)
sample = sample.reindex(columns=xgb_model.feature_names_in_, fill_value=0)

print(f"✅ Data prepared: {sample.shape[1]} features")
print("\n📊 Summary of input data:")
print("─"*60)
for cfg in FEATURE_CONFIG[:7]:  # Show first 7 features
    field = cfg["field"]
    if field in values:
        display_format = cfg.get("display_format", "{}")
        if cfg.get("is_binary"):
            display_value = "YES" if values[field] == 1 else "NO"
        else:
            display_value = display_format.format(values[field])
        print(f"{cfg['label']}: {display_value}")


🔄 Preparing data for prediction...
✅ Data prepared: 64 features

📊 Summary of input data:
────────────────────────────────────────────────────────────
🎂 Age: 18.0
💼 Work Experience: 0.0
💰 Annual Income: $20,000
🏦 Requested Loan Amount: $40,000
📊 Loan-to-Income Ratio: 2.0
💳 Credit Usage Rate: 0%
⏰ Average Days Late: 0.0


In [14]:
# Make prediction
print("\n🤖 Running XGBoost model prediction...")

# Get probability
prob = float(xgb_model.predict_proba(sample)[:, 1][0])

# Make decision
is_risky = prob >= threshold

# Determine risk level
if prob < 0.3:
    risk_level = "VERY LOW"
    risk_emoji = "🟢"
    risk_color = "GREEN"
elif prob < 0.5:
    risk_level = "LOW"
    risk_emoji = "🟡"
    risk_color = "YELLOW"
elif prob < 0.7:
    risk_level = "MEDIUM"
    risk_emoji = "🟠"
    risk_color = "ORANGE"
elif prob < threshold:
    risk_level = "HIGH"
    risk_emoji = "🔴"
    risk_color = "RED"
else:
    risk_level = "CRITICAL"
    risk_emoji = "🚨"
    risk_color = "RED"

# Display results
print("\n" + "="*80)
print("🎯 PREDICTION RESULTS")
print("="*80)

print(f"\n{risk_emoji} Default Probability: {prob:.1%}")
print(f"{risk_emoji} Risk Level: {risk_level}")
print(f"🎚️  Decision Threshold: {threshold:.1%}")

print("\n" + "─"*80)
print("📋 FINAL DECISION")
print("─"*80)

if is_risky:
    print("\n❌ REJECT APPLICATION")
    print("\n⚠️  Reasons:")
    print(f"   • Default probability ({prob:.1%}) exceeds threshold ({threshold:.1%})")
    print(f"   • Risk level: {risk_level}")
    print("\n💡 Recommendations:")
    print("   • Request additional collateral")
    print("   • Suggest a smaller loan amount")
    print("   • Consider a co-signer")
    print("   • Offer financial counseling")
else:
    print("\n✅ APPROVE APPLICATION")
    print("\n🎉 Good news! This customer shows low default risk.")
    print("\n💡 Recommendations:")
    if prob < 0.3:
        print("   • Standard approval process")
        print("   • Competitive interest rate")
        print("   • Consider for premium services")
    else:
        print("   • Standard approval with monitoring")
        print("   • Set up payment reminders")
        print("   • Regular credit review")

print("\n" + "="*80)
print("📊 MODEL INFORMATION")
print("="*80)
print(f"Model: XGBoost Classifier")
print(f"Threshold: {threshold:.3f}")
print(f"Expected Precision: 20.17%")
print(f"Expected Recall: 36.56%")
print(f"ROC-AUC Score: 0.7042")
print("="*80 + "\n")


🤖 Running XGBoost model prediction...

🎯 PREDICTION RESULTS

🟢 Default Probability: 12.9%
🟢 Risk Level: VERY LOW
🎚️  Decision Threshold: 86.0%

────────────────────────────────────────────────────────────────────────────────
📋 FINAL DECISION
────────────────────────────────────────────────────────────────────────────────

✅ APPROVE APPLICATION

🎉 Good news! This customer shows low default risk.

💡 Recommendations:
   • Standard approval process
   • Competitive interest rate
   • Consider for premium services

📊 MODEL INFORMATION
Model: XGBoost Classifier
Threshold: 0.860
Expected Precision: 20.17%
Expected Recall: 36.56%
ROC-AUC Score: 0.7042



In [12]:
# Optional: Show feature importance for this prediction
print("\n📈 TOP 10 FACTORS INFLUENCING THIS DECISION:")
print("─"*60)

# Get feature importance from XGBoost
feature_importance = pd.DataFrame({
    'feature': xgb_model.feature_names_in_,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Map feature names to friendly labels
feature_labels = {cfg['field']: cfg['label'] for cfg in FEATURE_CONFIG}

print("\nMost important features in the model:")
for idx, row in feature_importance.head(10).iterrows():
    feature = row['feature']
    importance = row['importance']
    label = feature_labels.get(feature, feature)
    
    # Get customer's value for this feature
    customer_value = values.get(feature, sample[feature].iloc[0])
    
    print(f"{idx+1}. {label}")
    print(f"   Importance: {importance:.4f} | Your value: {customer_value:.2f}")

print("\n✅ Prediction complete!\n")


📈 TOP 10 FACTORS INFLUENCING THIS DECISION:
────────────────────────────────────────────────────────────

Most important features in the model:
20. 💳 Credit Usage Rate
   Importance: 4.0000 | Your value: 0.00
13. raw_days_employed
   Importance: 3.0000 | Your value: 0.00
21. active_loans_count
   Importance: 3.0000 | Your value: 0.00
45. dpd_max
   Importance: 3.0000 | Your value: 0.00
30. 💳 Credit Card Usage
   Importance: 2.0000 | Your value: 42.00
22. closed_loans_count
   Importance: 2.0000 | Your value: 0.00
9. raw_credit_amt
   Importance: 2.0000 | Your value: 0.00
1. 🎂 Age
   Importance: 2.0000 | Your value: 18.00
46. on_time_ratio
   Importance: 2.0000 | Your value: 0.00
11. raw_goods_price
   Importance: 1.0000 | Your value: 0.00

✅ Prediction complete!

